# Xarray Example of Graph Execution

## Graph Execution

To use a different backend for the execution, pass the required backend to the `Fluent` constructor. Currently supported backends include `array_api_compat`, which is the default, `xarray` and `jax`.

In [ ]:
import numpy as np 
import xarray as xr
from cascade.fluent import Fluent, Payload
from cascade.backends.xarray_backend import XArrayBackend

payloads = np.empty((4, 5), dtype=object)
payloads[:] = Payload(xr.DataArray, [np.random.rand(2, 3)], {"dims": ["a", "b"]})
graph = (
    Fluent(backend=XArrayBackend).source(payloads, ["x", "y"])
    .mean("x")
    .minimum("y")
    .expand("z", 3, 1, 0)
    .map([Payload(lambda x, a=a: x * a) for a in range(1, 4)])
    .map(Payload(lambda x: print("RESULT", x)))
    .graph()
)

At the moment, the graphs can be executed using Dask but the Dask dynamic scheduling.

In [ ]:
import os
from cascade.executor import DaskExecutor
from cascade.scheduler import Schedule

os.environ["DASK_LOGGING__DISTRIBUTED"]="debug"
schedule = Schedule(graph, None, {})
executor = DaskExecutor(schedule)
executor.execute(memory_limit="10GB", n_workers=1, threads_per_worker=1)